In [2]:
## Langchian componet Imports
from langchain_community.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings

In [13]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
## Support for dataset retrival with Huggin face
from datasets import load_dataset

## With Cassio the engibne powering tje AstraDB integration in Langcahin
import cassio

/home/amitkumar/projects/GenAI/pdf_query_langchain/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from PyPDF2 import PdfReader

In [7]:
from typing_extensions import Concatenate

## API Keys and Tokens

In [5]:
ASTRA_DB_APPLICATION_TOKEN =""
ASTRA_DB_ID=""

OPENAI_API_KEY=""

In [6]:
## Convert the pdf into a readable format
pdfreader = PdfReader("COI...pdf")

In [8]:
## read text from pdf
raw_text = ""
for i,page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content
        

## Intialize the connection to database

In [10]:
## Intialize connection with Astradb
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

## Create langchain embeddings and LLM objects

In [11]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

## Creating Langchain vector store backed by AstraDB 

In [16]:
astra_vector_store= Cassandra(
    embedding=embedding,
    table_name="consti_qa",
    session=None,
    keyspace=None,
)

## Dividing the entire data into smaller chunks for and text embedding while inserting.

In [14]:
## split the text using character text Split such that it shold not increase token size.
text_splitter = CharacterTextSplitter(
    separator= "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,
)

texts = text_splitter.split_text(raw_text)

In [15]:
texts[:50]

['THE CONSTITUTION OF INDIA  \n [As on 26th November , 2021 ] \n \n \n  2 \n   \nPREFACE  \n \nThis is the fifth   pocket size edition of the Constitution of India in the \ndiglot form. In this edition, the text of the Constitution of India has been \nbrought up -to-date by incorporating therein all the amendments up to the \nConstitution (One Hundred and Fifth Amendment) A ct, 2021. The foot notes \nbelow the text indicate the Constitution Amendment Acts by which such \namendments have been made.  \nThe Constitution (One Hundredth Amendments ) Act, 2015 containing \ndetails of acquired and transferred territories between the Governme nts of \nIndia and Bangladesh has been provided in APPENDIX I.  \nThe Constitution (Application to Jammu and Kashmir) Order, 2019 and',
 'India and Bangladesh has been provided in APPENDIX I.  \nThe Constitution (Application to Jammu and Kashmir) Order, 2019 and \nthe declaration under article 370(3) of the Constitution have been provided \nrespectively i

## Load the dataset into the vector space 

In [18]:
astra_vector_store.add_texts(texts)
print("Inserted %i headline." % len(texts))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 1444 headline.


## Run the QA cycle

In [19]:
first_question = True
while True:
    if first_question:
        query_text = input("\n Enter your Question(or type 'quit' to 'exit'): ").strip()
    else:
        query_text = input("\n Whta's your next Question(or type 'quit' to 'exit'): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    print("\n QUESTION: \"%s\"" % query_text)

    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER : \"%s\"\n" % answer)

    print("FIRST DOCUMNETS BY RELEVANCE:")
    for doc , score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("     [%0.4f]\"%s...\"" % (score, doc.page_content[:84]))


 QUESTION: "Tell me about the fundamental rights"
ANSWER : "The fundamental rights are a set of rights granted to all citizens of India by the Constitution. These rights protect individuals from any unfair treatment or discrimination by the government or other authorities. The fundamental rights include the right to equality, right to freedom, right against exploitation, and right to freedom of religion. They also include cultural and educational rights, such as the right to establish and administer educational institutions. These rights are enforceable by law and any law that goes against them is considered void."

FIRST DOCUMNETS BY RELEVANCE:
     [0.9251]"13. Laws inconsistent with or in derogation of the fundamental 
rights.  
 Right to ..."
     [0.9251]"13. Laws inconsistent with or in derogation of the fundamental 
rights.  
 Right to ..."
     [0.9215]"termination of citizenship and all other matters relating to citizenship.  
 37 
 PA..."
     [0.9179]"Right against Exploita